Import packages needed

In [1]:
import suite2p
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from natsort import natsorted

Pre-set parameters, and load the only bin files in the pre-set path folder

In [28]:
opsSetfile=r'C:\Users\zhangl33\Projects\Project4-SLMonline\LuSLMOnlineTest\ops3Planes.npy';
ops0 = np.load(opsSetfile,allow_pickle=True).item()
ops0['input_format']='binary';
#ops0['save_path0']='C:/Users/zhangl33/Projects/Project4-SLMonline/LuSLMOnlineTest/02082024/'
ops0['save_path0']=r'C:\Users\zhangl33\Projects\Project4-SLMonline\LuSLMOnlineTest\02082024\\'
ops0['Lx']=512  ##Width of a image
ops0['Ly']=512  ##Height of a image
ops0['xrange']=[0,512]
ops0['yrange']=[0,512]
ops0['nplanes']=3
ops0['nchannels']=1
ops0['tau']=0.9
ops0['fs']=10;
nplanes=ops0['nplanes'];
binFile = glob.glob(ops0['save_path0'] + '/*.bin')
print(binFile)


## creat new folder to process the data

SaveFolder=os.path.join(ops0['save_path0'], 'binProcessed');
print(SaveFolder)
if not os.path.exists(SaveFolder):
   os.makedirs(SaveFolder)
else:
   print("Folder already exists.")

['C:\\Users\\zhangl33\\Projects\\Project4-SLMonline\\LuSLMOnlineTest\\02082024\\TSeries-02082024-001.bin']
C:\Users\zhangl33\Projects\Project4-SLMonline\LuSLMOnlineTest\02082024\\binProcessed
Folder already exists.


Load bin files

In [30]:

os.path.exists(binFile[0])
if len(binFile)>1:
   print('More than one bin files found!!')
else:
   rawBin = suite2p.io.BinaryFile(Ly=ops0['Ly'],Lx=ops0['Lx'], filename=binFile[0])
   nFrame = rawBin.shape[0] // nplanes


Suite2p processing with no motion correction

In [44]:
ops1=ops0
#ops1['save_path']='C:/Users/zhangl33/Projects/Project4-SLMonline/LuSLMOnlineTest/02082024/CommandLineProcess/'
for plane_idx in range(nplanes):
    print('Processing plane ' + str(plane_idx))
   # fpath = ops1["save_path"]
    fpath=os.path.join(SaveFolder, f'plane{plane_idx}/');
    if not os.path.exists(fpath):
       os.makedirs(fpath)
    else:
       print("Saving folder already exists.")
    
    plane_data = rawBin[range(0+plane_idx,rawBin.shape[0],nplanes),:,:]
    ops, stat = suite2p.detection_wrapper(f_reg=plane_data, ops=ops1)
    ops['nframes']=nFrame
    ops['meanImg']=np.mean(plane_data,axis=0)
    ops=suite2p.registration.register.enhanced_mean_image(ops)
    np.save(os.path.join(fpath, "ops.npy"), ops)
    stat_after_extraction, F, Fneu, F_chan2, Fneu_chan2 = suite2p.extraction_wrapper(stat, f_reg=plane_data,
                                                                   f_reg_chan2 = plane_data,ops=ops1)
    np.save(os.path.join(fpath, "ops.npy"), ops)
    if len(stat) > 0:
       classfile=suite2p.classification.builtin_classfile
       iscell = suite2p.classify(stat=stat_after_extraction, classfile=classfile)

# Apply preprocessing step for deconvolution
       dF = F.copy() - ops['neucoeff']*Fneu

       dF = suite2p.extraction.preprocess(
            F=dF,
            baseline=ops1['baseline'],
            win_baseline=ops1['win_baseline'],
            sig_baseline=ops1['sig_baseline'],
            fs=ops1['fs'],
            prctile_baseline=ops1['prctile_baseline']
           )
# Identify spikes
       spks = suite2p.extraction.oasis(F=dF, batch_size=ops1['batch_size'], tau=ops['tau'], fs=ops['fs'])
       #ops=ops1;
       stat=stat
       np.save(os.path.join(fpath, "stat.npy"), stat)
       np.save(os.path.join(fpath, "F.npy"), F)
       np.save(os.path.join(fpath, "Fneu.npy"), Fneu)
       np.save(os.path.join(fpath, "iscell.npy"), iscell)
       np.save(os.path.join(fpath, "ops.npy"), ops)
       np.save(os.path.join(fpath, "spks.npy"), spks)

       ops_matlab=ops;
       ops_matlab['save_path']=fpath;
       suite2p.io.save_mat(ops, stat, F, Fneu, spks, iscell,redcell=-1)
    else:
       print("no ROIs found, only ops.npy file saved") 

Processing plane 0
Saving folder already exists.
Binning movie in chunks of length 09
Binned movie of size [500,512,512] created in 2.88 sec.
Binned movie denoised (for cell detection only) in 11.40 sec.
NOTE: FORCED spatial scale ~6 pixels, time epochs 1.00, threshold 10.00 
0 ROIs, score=78.31
Detected 93 ROIs, 6.07 sec
After removing overlaps, 88 ROIs remain
added enhanced mean image
Masks created, 0.18 sec.
Extracted fluorescence from 88 ROIs in 4500 frames, 3.42 sec.
Extracted fluorescence from 88 ROIs in 4500 frames, 3.40 sec.
['npix_norm', 'skew', 'compact']
Processing plane 1
Saving folder already exists.
Binning movie in chunks of length 09
Binned movie of size [500,512,512] created in 2.94 sec.
Binned movie denoised (for cell detection only) in 11.77 sec.
NOTE: FORCED spatial scale ~6 pixels, time epochs 1.00, threshold 10.00 
0 ROIs, score=59.51
Detected 60 ROIs, 5.89 sec
After removing overlaps, 60 ROIs remain
added enhanced mean image
Masks created, 0.20 sec.
Extracted flu

In [ ]:
If there are more than 1 plane, combine all planes data into one

In [43]:
#suite2p.io.save.combined(ops1['save_path'],save=True)
#suite2p.io.combined(ops1['save_path'],save=True)
#ops1=ops0
#ops1['save_path']='C:/Users/zhangl33/Projects/Project4-SLMonline/LuSLMOnlineTest/02082024/CommandLineProcess/'
if ops0['nplanes']>1:
   result=suite2p.io.save.combined(SaveFolder,save=True)
   stat, ops, F, Fneu, spks, iscell_0, iscell_1, redcell_0, redcell_1, hasred = result
   iscell = np.stack((iscell_0, iscell_1), axis=-1)
   #print(iscell)
   fpath=os.path.join(SaveFolder, 'combined');
if not os.path.exists(fpath):
   os.makedirs(fpath)
else:
   print("Folder already exists.")

   np.save(os.path.join(fpath, "F.npy"), F)
   np.save(os.path.join(fpath, "Fneu.npy"), Fneu)
   np.save(os.path.join(fpath, "spks.npy"), spks)
   np.save(os.path.join(fpath, "ops.npy"), ops)
   np.save(os.path.join(fpath, "stat.npy"), stat)
   np.save(os.path.join(fpath, "iscell.npy"), iscell)
   ops_matlab=ops;
   ops_matlab['save_path']=fpath;
   suite2p.io.save_mat(ops, stat, F, Fneu, spks, iscell,redcell=-1)

  

appended plane 0 to combined view
appended plane 1 to combined view
appended plane 2 to combined view
Folder already exists.
